In [42]:
import seaborn as sns
import pandas as pd
import numpy as np

In [43]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# GROUPBY

In [44]:
# After creating a GroupBy object, three types of operations can be performed on them
tips_gb = tips.groupby(["sex", "smoker"])
tips_gb

# AGG

The aggregate operation aggregates all the data in these groups into one value. You use a dictionary to specify which values you'd like. For example look below, we are aksing for both the mean and the min value of the tip column for each group:
1. mean(): Compute mean of groups
2. sum(): Compute sum of group values
3. size(): Compute group sizes
4. count(): Compute count of group
5. std(): Standard deviation of groups
6. var(): Compute variance of groups
7. sem(): Standard error of the mean of groups
8. describe(): Generates descriptive statistics
9. first(): Compute first of group values
10. last(): Compute last of group values
11. nth() : Take nth value, or a subset if n is a list
12. min(): Compute min of group values
13. max(): Compute max of group values

In [45]:
# aggregate
tips_agg = tips_gb.agg({
    "tip" : ["mean", "min", "size"],
    "day" : ["first", "last"],
    "total_bill" : ["size", "max", "sum"]
})

tips_agg

tip              day       total_bill                
                   mean   min size first  last       size    max      sum
sex    smoker                                                            
Male   Yes     3.051167  1.00   60   Sat   Sat         60  50.81  1337.07
       No      3.113402  1.25   97   Sun   Sat         97  48.33  1919.75
Female Yes     2.931515  1.00   33   Sat   Sat         33  44.30   593.27
       No      2.773519  1.00   54   Sun  Thur         54  35.83   977.68

In [46]:
tips_agg.reset_index()

sex smoker       tip              day       total_bill                
                      mean   min size first  last       size    max      sum
0    Male    Yes  3.051167  1.00   60   Sat   Sat         60  50.81  1337.07
1    Male     No  3.113402  1.25   97   Sun   Sat         97  48.33  1919.75
2  Female    Yes  2.931515  1.00   33   Sat   Sat         33  44.30   593.27
3  Female     No  2.773519  1.00   54   Sun  Thur         54  35.83   977.68

In [47]:
tips_agg.columns

MultiIndex([(       'tip',  'mean'),
            (       'tip',   'min'),
            (       'tip',  'size'),
            (       'day', 'first'),
            (       'day',  'last'),
            ('total_bill',  'size'),
            ('total_bill',   'max'),
            ('total_bill',   'sum')],
           )

In [48]:
tips_agg.stack()

day        tip  total_bill
sex    smoker                                   
Male   Yes    first   Sat        NaN         NaN
              last    Sat        NaN         NaN
              max     NaN        NaN       50.81
              mean    NaN   3.051167         NaN
              min     NaN   1.000000         NaN
              size    NaN  60.000000       60.00
              sum     NaN        NaN     1337.07
       No     first   Sun        NaN         NaN
              last    Sat        NaN         NaN
              max     NaN        NaN       48.33
              mean    NaN   3.113402         NaN
              min     NaN   1.250000         NaN
              size    NaN  97.000000       97.00
              sum     NaN        NaN     1919.75
Female Yes    first   Sat        NaN         NaN
              last    Sat        NaN         NaN
              max     NaN        NaN       44.30
              mean    NaN   2.931515         NaN
              min     NaN   1.000000         NaN
              size    NaN  33.000000       33.00
              sum     NaN        NaN      593.27
       No     first   Sun        NaN         NaN
              last   Thur        NaN         NaN
              max     NaN        NaN       35.83
              mean    NaN   2.773519         NaN
              min     NaN   1.000000         NaN
              size    NaN  54.000000       54.00
              sum     NaN        NaN      977.68

In [49]:
tips_agg.columns.values

array([('tip', 'mean'), ('tip', 'min'), ('tip', 'size'), ('day', 'first'),
       ('day', 'last'), ('total_bill', 'size'), ('total_bill', 'max'),
       ('total_bill', 'sum')], dtype=object)

In [50]:
"__".join(tips_agg.columns.values[0])

'tip__mean'

In [51]:
"__".join(tips_agg.columns.values[0]).strip()

'tip__mean'

In [52]:
tips_agg.columns = ["__".join(col).strip() for col in tips_agg.columns.values]
tips_agg.columns

Index(['tip__mean', 'tip__min', 'tip__size', 'day__first', 'day__last',
       'total_bill__size', 'total_bill__max', 'total_bill__sum'],
      dtype='object')

In [53]:
tips_agg

tip__mean  tip__min  tip__size day__first day__last  \
sex    smoker                                                        
Male   Yes      3.051167      1.00         60        Sat       Sat   
       No       3.113402      1.25         97        Sun       Sat   
Female Yes      2.931515      1.00         33        Sat       Sat   
       No       2.773519      1.00         54        Sun      Thur   

               total_bill__size  total_bill__max  total_bill__sum  
sex    smoker                                                      
Male   Yes                   60            50.81          1337.07  
       No                    97            48.33          1919.75  
Female Yes                   33            44.30           593.27  
       No                    54            35.83           977.68

# FILTER

In [55]:
tips_gb = tips.groupby(["day", "time"])

In [58]:
tips_gb.agg({"size":"sum"}).median()

size    88.0
dtype: float64

In [59]:
median_size = tips_gb.agg({"size":"sum"}).median()[0]

In [63]:
median_size

88.0

In [65]:
tips_gb["size"].sum()

day   time  
Thur  Lunch     150.0
      Dinner      2.0
Fri   Lunch      14.0
      Dinner     26.0
Sat   Lunch       NaN
      Dinner    219.0
Sun   Lunch       NaN
      Dinner    216.0
Name: size, dtype: float64

In [66]:
tips_gb.filter(lambda group: group["size"].sum() < median_size)

,total_bill,tip,sex,smoker,day,time,size
90,28.97,3.00,Male,Yes,Fri,Dinner,2
91,22.49,3.50,Male,No,Fri,Dinner,2
92,5.75,1.00,Female,Yes,Fri,Dinner,2
93,16.32,4.30,Female,Yes,Fri,Dinner,2
94,22.75,3.25,Female,No,Fri,Dinner,2
95,40.17,4.73,Male,Yes,Fri,Dinner,4
96,27.28,4.00,Male,Yes,Fri,Dinner,2
97,12.03,1.50,Male,Yes,Fri,Dinner,2
98,21.01,3.00,Male,Yes,Fri,Dinner,2
99,12.46,1.50,Male,No,Fri,Dinner,2


In [68]:
tips_gb.filter(lambda group: group["total_bill"].sum() < 100)

,total_bill,tip,sex,smoker,day,time,size
220,12.16,2.20,Male,Yes,Fri,Lunch,2
221,13.42,3.48,Female,Yes,Fri,Lunch,2
222,8.58,1.92,Male,Yes,Fri,Lunch,1
223,15.98,3.00,Female,No,Fri,Lunch,3
224,13.42,1.58,Male,Yes,Fri,Lunch,2
225,16.27,2.50,Female,Yes,Fri,Lunch,2
226,10.09,2.00,Female,Yes,Fri,Lunch,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [74]:
tips_gb.filter(lambda group: group["total_bill"].max() < 25)

,total_bill,tip,sex,smoker,day,time,size
220,12.16,2.20,Male,Yes,Fri,Lunch,2
221,13.42,3.48,Female,Yes,Fri,Lunch,2
222,8.58,1.92,Male,Yes,Fri,Lunch,1
223,15.98,3.00,Female,No,Fri,Lunch,3
224,13.42,1.58,Male,Yes,Fri,Lunch,2
225,16.27,2.50,Female,Yes,Fri,Lunch,2
226,10.09,2.00,Female,Yes,Fri,Lunch,2
243,18.78,3.00,Female,No,Thur,Dinner,2


# TRANSFORM

In [81]:
tips_gb = tips.groupby(["day"])

In [79]:
tips.groupby(["day"]).mean()

,total_bill,tip,size
day,,,
Thur,17.682742,2.771452,2.451613
Fri,17.151579,2.734737,2.105263
Sat,20.441379,2.993103,2.517241
Sun,21.410000,3.255132,2.842105


In [82]:
tips_gb[["total_bill", "tip"]].transform(lambda x : x / x.mean()).head()

,total_bill,tip
0,0.793554,0.310279
1,0.482952,0.509964
2,0.981317,1.075225
3,1.106025,1.016856
4,1.148529,1.109018


In [84]:
tips_gb[["total_bill"]].transform(lambda x : x / x.mean() * 10).head()

,total_bill
0,7.935544
1,4.829519
2,9.813171
3,11.060252
4,11.485287
